# MNIST

## Import Libraries

In [11]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim

## Prepare the dataset

### Load the dataset

In [12]:
train_dataset = datasets.MNIST('./data', train=True, download=True,
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                                   ]))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.MNIST('./data', train=False,
                             transform=transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.1307,), (0.3081,))
                                 ]))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

## Model(LeNet-5)

In [13]:
class LeNet5_1(nn.Module):
    def __init__(self):
        super(LeNet5_1, self).__init__()
        self.conv_1 = nn.Conv2d(1, 6, kernel_size=5, stride=1)
        self.avgpool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv_2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.conv_3 = nn.Conv2d(16, 120, kernel_size=5, stride=1)
        self.tanh = nn.Tanh()
        self.fc_1 = nn.Linear(120, 84)
        self.fc_2 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv_1(x)
        x = self.tanh(x)
        x = self.avgpool(x)
        x = self.conv_2(x)
        x = self.tanh(x)
        x = self.avgpool(x)
        x = self.conv_3(x)
        x = self.tanh(x)
        x = self.fc_1(x)
        x = self.tanh(x)
        res = self.fc_2(x)
        return res

In [14]:
class LeNet5_2(nn.Module):
    def __init__(self):
        super(LeNet5_2, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 6, 5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(2, stride=2),
            nn.Conv2d(6, 16, 5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(2, stride=2),
            nn.Conv2d(16, 120, 5, stride=1),
            nn.Tanh(),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(120, 84),
            nn.Tanh(),
            nn.Linear(84, 10)
        )
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 120)
        x = self.fc_layers(x)
        return x

## Main

In [15]:
batch_size = 64
epochs = 1000

model = LeNet5_2()

optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_function = nn.CrossEntropyLoss()